# Nanodegree Engenheiro de Machine Learning
## Aprendizagem Supervisionada
## Project 2: Construindo um Sistema de Intervenção para Estudantes

Bem-vindo ao segundo projeto do Nanodegree de Machine Learning! Neste Notebook, alguns templates de código já foram fornecidos, e será o seu trabalho implementar funcionalidades necessárias para completar este projeto com êxito. Seções que começam com **'Implementação'** no cabeçalho indicam que o bloco de código que se segue precisará de funcionalidades adicionais que você deve fornecer. Instruções serão providenciadas para cada seção e as especificações para cada implementação estarão marcadas no bloco de código com o comando `'TODO'`. Tenha certeza de ler atentamente todas as instruções!

Além do código implementado, haverá questões relacionadas ao projeto e à implementação que você deve responder. Cada seção em que você tem que responder uma questão será antecedida de um cabeçalho **'Questão X'**. Leia atentamente cada questão e escreva respostas completas nas caixas de texto subsequentes que começam com **'Resposta: '**. O projeto enviado será avaliado baseado nas respostas para cada questão e a implementação que você forneceu.  

>**Nota:** Células de código e Markdown podem ser executadas utilizando o atalho de teclado **Shift + Enter**. Além disso, as células Markdown podem ser editadas, um clique duplo na célula entra no modo de edição.

### Questão 1 - Classificação versus Regressão
*Seu objetivo neste projeto é identificar estudantes que possam precisar de intervenção antecipada antes de serem reprovados. Que tipo de problema de aprendizagem supervisionada é esse: classificação ou regressão? Por quê?*

**Resposta: ** A classificação é utilizada quando temos resultados discretos, ou seja, os valores finais são finitos. Na regressão, os valores numéricos correspondem a uma escala contínua, infinitos. 

Neste problema é necessário identificar que temos dois resultados possíveis: reprovado ou aprovado. Assim este é um problema de aprendizagem supervisionada de **classificação**.


## Observando os Dados
Execute a célula de código abaixo para carregar as bibliotecas de Python necessárias e os dados sobre os estudantes. Note que a última coluna desse conjunto de dados, `'passed'`, será nosso rótulo alvo (se o aluno foi ou não aprovado). As outras colunas são atributos sobre cada aluno.

In [3]:
# Importar bibliotecas
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score

# Ler os dados dos estudantes
student_data = pd.read_csv("student-data.csv")
print ("Os dados dos estudantes foram lidos com êxito!")

Os dados dos estudantes foram lidos com êxito!


### Implementação: Observando os Dados
Vamos começar observando o conjunto de dados para determinar quantos são os estudantes sobre os quais temos informações e entender a taxa de graduação entre esses estudantes. Na célula de código abaixo, você vai precisar calcular o seguinte:
- O número total de estudantes, `n_students`.
- O número total de atributos para cada estudante, `n_features`.
- O número de estudantes aprovados, `n_passed`.
- O número de estudantes reprovados, `n_failed`.
- A taxa de graduação da classe, `grad_rate`, em porcentagem (%).


In [2]:
n_students = student_data.shape[0]

n_features = student_data.shape[1]-1

n_passed = student_data[(student_data.passed == 'yes')].shape[0]

n_failed = student_data[(student_data.passed == 'no')].shape[0]

grad_rate = (float(n_passed) / float(n_students))*100 

print("Número total de estudantes: {}".format(n_students))
print("Número de atributos: {}".format(n_features))
print("Número de estudantes aprovados: {}".format(n_passed))
print("Número de estudantes reprovados: {}".format(n_failed))
print("Taxa de graduação: {:.2f}%".format(grad_rate))


Número total de estudantes: 395
Número de atributos: 30
Número de estudantes aprovados: 265
Número de estudantes reprovados: 130
Taxa de graduação: 67.09%


## Preparando os Dados
Nesta seção, vamos preparara os dados para modelagem, treinamento e teste.

### Identificar atributos e variáveis-alvo
É comum que os dados que você obteve contenham atributos não numéricos. Isso pode ser um problema, dado que a maioria dos algoritmos de machine learning esperam dados númericos para operar cálculos.

Execute a célula de código abaixo para separar os dados dos estudantes em atributos e variáveis-alvo e verificar se algum desses atributos é não numérico.

In [1]:
# Extraia as colunas dos atributo
feature_cols = list(student_data.columns)

# Extraia a coluna-alvo, 'passed'
target_col = student_data.columns[-1] 

# Mostre a lista de colunas
print "Colunas de atributos:\n{}".format(feature_cols)
print "\nColuna-alvo: {}".format(target_col)

# Separe os dados em atributos e variáveis-alvo (X_all e y_all, respectivamente)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Mostre os atributos imprimindo as cinco primeiras linhas
print "\nFeature values:"
print X_all.head()

SyntaxError: invalid syntax (<ipython-input-1-b1765cd70a4e>, line 8)

Estudando um pouco as informações dos atributos:<br>

| Atributo | Tipo | Descrição |
| :------ | :-- | :------- | 
|'school'| binário| Indica a escola do estudante (‘GP’ - Gabriel Pereira, ‘MS’ - Mousinho da Silveira)|
|'sex'| binário| ‘F’ - Female or 'M' - Male|
|'age'| numérico| Idade do estudante (de 15 a 22 anos)|
|'address'| binário| Tipo do endereço da casa do estudante ('U' - urbano ou 'R' - rural)|
|'famsize'| binário| Tamanho da família (LE3 - menos ou igual a 3, GT3 - maior que 3)|
|'Pstatus'| binário| 'T' - Pais morando juntos ou 'A' - Pais separados)|
|'Medu'| numérico| Nível de educação da mãe, (0 - nenhum, 1 - educação primária (4ª série), 2 - 5ª a 9ª série, 3 - educação secundária ou 4 - ensino superior)|
|'Fedu'| numérico| Nível de educação do pai, (0 - nenhum, 1 - educação primária (4ª série), 2 - 5ª a 9ª série, 3 - educação secundária ou 4 - ensino superior)|
|'Mjob'| nominal| Trabalho da mãe ('teacher' - professora, 'health' relacionado a saúde, civil 'services' (exemplo, administrativo ou polícia), 'at_home' (em casa) or 'other' (outros))|
|'Fjob'| nominal| Trabalho do pai ('teacher' - professor, 'health' relacionado a saúde, civil 'services'  (serviços civis, exemplo, administrativo ou polícia), 'at_home' (em casa) or 'other' (outros))|
|'reason'| nominal| Razão de escolha da escola(close to 'home' - perto de casa, school 'reputation' - reputação da escola, 'course' preference - preferência do curso, ou 'other' - outros)|
|'guardian' | nominal| Guarda do aluno ('mother' - mãe, 'father' - pai ou 'other' - outros)|
|'traveltime' | numérico | Tempo de viagem de casa até a escola (1 - <15 min., 2 - 15 a 30 min., 3 - 30 min. a 1 hora, ou 4 - >1 hora)|
|'studytime'| numérico| Tempo semanal de estudo (1 - <2 horas, 2 - 2 a 5 horas, 3 - 5 a 10 horas, ou 4 - >10 horas)|
|'failures'| numérico| Número de faltas em classes anteriores (n if 1<=n<3, else 4)|
|'schoolsup'| binário| Suporte educacional extra (yes/no)|
|'famsup'| binário| Suporte educacional da família (yes/no)|
|'paid'| binário| Pagamento extra de aulas (matemática ou português) (yes/no)|
|'activities'| binário| Atividades extracurriculares (yes/no)|
|'nursery'| binário| Frequentou escola maternal (yes/no)|
|'higher'| binário| Quer fazer curso superior (yes/no)|
|'internet'| binário| Possui acesso a internet em casa (yes/no)|
|'romantic'| binário| Possui relacionamento romântico (yes/no)|
|'famrel'| numérico| Qualidade do relacionamento familiar (de 1 - muito ruim a 5 - excelente)|
|'freetime'| numérico| tempo livre depois da escola (de 1 - muito baixo a 5 - muito alto)|
|'goout'| numérico| Sai com os amigos (de 1 - pouco a 5 - muito)|
|'Dalc'| numérico| Consumo de álcool em dia de semana (de 1 - muito baixo a 5 - muito alto)|
|'Walc'| numérico| Consumo de álcool em fim de semana (de 1 - muito baixo a 5 - muito alto)|
|'health'| numérico| Atual estado de saúde (de 1 - muito ruim a 5 - muito bom)|
|'absences'| numérico|Ausências escolares (de 0 a 93)|
|'passed'| binário| Indica se o aluno passou (yes/no)|


### Pré-processar Colunas de Atributo

Como você pode ver, há muitas colunas não numéricas que precisam ser convertidas! Muitas delas são simplesmente `yes`/`no`, por exemplo, a coluna `internet`. É razoável converter essas variáveis em valores (binários) `1`/`0`.

Outras colunas, como `Mjob` e `Fjob`, têm mais do que dois valores e são conhecidas como variáveis categóricas. A maneira recomendada de lidar com esse tipo de coluna é criar uma quantidade de colunas proporcional aos possíveis valores (por exemplo, `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc), e assinalar `1` para um deles e `0` para todos os outros.

Essas colunas geradas são por vezes chamadas de _variáveis postiças_ (em inglês: _dummy variables_), e nós iremos utilizar a função [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) para fazer essa conversão. Execute a célula de código abaixo para executar a rotina de pré-processamento discutida nesta seção.

In [4]:
def preprocess_features(X):
    ''' Pré-processa os dados dos estudantes e converte as variáveis binárias não numéricas em
        variáveis binárias (0/1). Converte variáveis categóricas em variáveis postiças. '''
    
    # Inicialize nova saída DataFrame
    output = pd.DataFrame(index = X.index)

    # Observe os dados em cada coluna de atributos 
    for col, col_data in X.iteritems():
        
        # Se o tipo de dado for não numérico, substitua todos os valores yes/no por 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # Se o tipo de dado for categórico, converta-o para uma variável dummy
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Reúna as colunas revisadas
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))

Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementação: Divisão dos Dados de Treinamento e Teste
Até agora, nós convertemos todos os atributos _categóricos_ em valores numéricos. Para o próximo passo, vamos dividir os dados (tanto atributos como os rótulos correspondentes) em conjuntos de treinamento e teste. Na célula de código abaixo, você irá precisar implementar o seguinte:
- Embaralhe aleatoriamente os dados (`X_all`, `y_all`) em subconjuntos de treinamento e teste.
  - Utilizar 300 pontos de treinamento (aproxidamente 75%) e 95 pontos de teste (aproximadamente 25%).
  - Estabelecer um `random_state` para as funções que você utiliza, se a opção existir.
  - Armazene os resultados em `X_train`, `X_test`, `y_train` e `y_test`.

In [5]:
# TODO: Importe qualquer funcionalidade adicional de que você possa precisar aqui
from sklearn.model_selection import train_test_split


# TODO: Estabeleça o número de pontos de treinamento
num_train = 300

# Estabeleça o número de pontos de teste
#num_test = X_all.shape[0] - num_train


# TODO: Emabaralhe e distribua o conjunto de dados de acordo com o número de pontos de treinamento e teste abaixo
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=42)

# Mostre o resultado da distribuição
print "O conjunto de treinamento tem {} amostras.".format(X_train.shape[0])
print "O conjunto de teste tem {} amostras.".format(X_test.shape[0])

O conjunto de treinamento tem 300 amostras.
O conjunto de teste tem 95 amostras.


## Treinando e Avaliando Modelos
Nesta seção, você irá escolher 3 modelos de aprendizagem supervisionada que sejam apropriados para esse problema e que estejam disponíveis no `scikit-learn`. Primeiro você irá discutir o raciocínio por trás da escolha desses três modelos considerando suas vantagens e desvantagens e o que você sabe sobre os dados. Depois você irá ajustar o modelo a diferentes tamanhos de conjuntos de treinamento (com 100, 200 e 300 pontos) e medir a pontuação F<sub>1</sub>. Você vai precisar preencher três tabelas (uma para cada modelo) que mostrem o tamanho do conjunto de treinamento, o tempo de treinamento, o tempo de previsão e a pontuação F<sub>1</sub> no conjunto de treinamento.

**Os seguintes modelos de aprendizagem supervisionada estão atualmente disponíveis no **[`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html)** para você escolher:**
- Gaussian Naive Bayes (GaussianNB)
- Árvores de Decisão
- Métodos de agregação (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Método do gradiente estocástico (SGDC)
- Máquinas de vetores de suporte (SVM)
- Regressão logística

### Questão 2 - Aplicação dos Modelos
*Liste três modelos de aprendizagem supervisionada que são apropriadas para esse problema. Para cada modelo escolhido:*
- Descreva uma aplicação em mundo real na indústria em que o modelo pode ser aplicado. *(Talvez você precise fazer um pouco de pesquisa para responder essa questão – dê as devidas referências!)* 
- Quais são as vantagens do modelo; quando ele tem desempenho melhor? 
- Quais são as desvantagens do modelo, quando ele tem desempenho pior?
- O que faz desse modelo um bom candidato para o problema, considerando o que você sabe sobre os dados?

**Resposta: **

**Linear Support Vector Classification**<br>

<u>Exemplo de utilização</u><br>
Melhorando a generalização de Linear Support Vector Machines: um aplicativo para reconhecimento de objetos 3D com fundo desordenado. http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.44.6752&rep=rep1&type=pdf

<u>Vantagens</u>
- Eficaz em espaços altamente dimensionais, que são modelos de conjuntos de dados com muitos atributos, pondendo ser representado onde cada registro é representado no espaço com sua posição. 
- Eficaz nos casos em que o número de dimensões é maior do que o número de amostras.
- Eficiente em memória, pois utiliza um subconjunto de pontos de treinamento na função de decisão (chamado de vetores de suporte).


<u>Desvantagem</u>
- Não funcionam muito bem quando o conjunto de dados for muito grande.
- O tempo de treinamento é o cúbico no tamanho do conjunto de dados.
- Também apresentam desempenho inferior quando há grandes quantidades de ruído nos dados.
- Quando não há uma margem de divisão clara, também não funcionam muito bem.

<u>Porque escolheria</u>
- É um algoritmo rápido e robusto.
- É uma das primeiras opções de algoritmos para classificação quando se tem menos de 100K de amostras.




**K-Nearest Neighbors**<br>

<u>Exemplo de utilização</u><br>
Aplicação do algoritmo K-nearest neighbors sobre problema de diagnóstico de câncer de mama. <br>
Neste paper o autor fala que o resultado de classificação geral 1,17% melhor que o melhor resultado conhecido por esse problema.<br>
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2243774/


<u>Vantagens</u>
 - Robusto mesmo em dados com ruídos.
 - Eficaz mesmo se os dados de treinamento forem grandes.
 - Sem custo computacional no treinamento.
 
<u>Desvantagem</u>
- Desempenho inferior com poucos dados de treinamento.
- O custo de computação e memória é bastante elevado porque precisamos calcular a distância de cada instância de consulta para todas as amostras de treinamento.
- O algoritmo apenas computa a distância entre os dados vizinhos, não "aprende" com os dados de treinamento, o que acaba não generalizando bem.

<u>Porque escolheria</u>
- Como o tempo de treinamento é baixo, é um bom candidato, pois o custo de computação não é alto.

**Random Forest**

<u>Exemplo de utilização</u><br>
Classificação de precursores reais e pseudo microRNA usando modelo de previsão Random Forest com características combinadas. Neste paper, os resultados sugerem que o método prevê uma especificidade de 98,21% e uma sensibilidade de 95,09%.<br>
Fonte: https://academic.oup.com/nar/article/35/suppl_2/W339/2923422

<u>Vantagens</u><br>
- Por usar várias árvores de decisão, possui um desempenho melhor do que apenas uma árvore de decisão.
- Ele pode lidar com milhares de variáveis de entrada sem exclusão variável.
- Mostra a estimativa de quais variáveis são mais importantes na classificação.

<u>Desvantagens</u><br>
- Overfitting em dados com muito ruído.
- Quando há várias features, porém somente alguns deles são úteis, o desempenho acaba sendo pior.
- Difente de apenas uma árvore, Random Forest não é tão fácil interpretar visualmente. 

**Porque utilizaria** <br>
- São vários atributos de entrada correlacionados, como ele trabalha bem com várias entradas, acredito que é uma boa escolha.



### Configuração
Execute a célula de código abaixo para inicializar três funções de ajuda que você pode utilizar para treinar e testar os três modelos de aprendizagem supervisionada que você escolheu acima. As funções são as seguintes:
- `train_classifier` - recebe como parâmetro um classificador e dados de treinamento e ajusta o classificador aos dados.
- `predict_labels` - recebe como parâmetro um classificador ajustado, atributos e rótulo alvo e faz estimativas utilizando a pontuação do F<sub>1</sub>.
- `train_predict` - recebe como entrada um classificador, e dados de treinamento e teste, e executa `train_clasifier` e `predict_labels`.
 - Essa função vai dar a pontuação F<sub>1</sub> tanto para os dados de treinamento como para os de teste, separadamente.

In [6]:
def train_classifier(clf, X_train, y_train):
    ''' Ajusta um classificador para os dados de treinamento. '''
    
    # Inicia o relógio, treina o classificador e, então, para o relógio
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Imprime os resultados
    print "O modelo foi treinado em {:.4f} segundos".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Faz uma estimativa utilizando um classificador ajustado baseado na pontuação F1. '''
    
    # Inicia o relógio, faz estimativas e, então, o relógio para
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Imprime os resultados de retorno
    print "As previsões foram feitas em {:.4f} segundos.".format(end - start)
    print "Accuracy Score {}".format(accuracy_score (target.values, y_pred))
    print "Recall Score {}".format(recall_score (target.values, y_pred, average=None))
    print "Precision Score {}".format(precision_score (target.values, y_pred, pos_label='yes'))

    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Treina e faz estimativas utilizando um classificador baseado na pontuação do F1. '''
    
    # Indica o tamanho do classificador e do conjunto de treinamento
    print "Treinando um {} com {} pontos de treinamento. . .".format(clf.__class__.__name__, len(X_train))
    
    # Treina o classificador
    train_classifier(clf, X_train, y_train)
    
    # Imprime os resultados das estimativas de ambos treinamento e teste
    print "Pontuação F1 para o conjunto de treino: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "Pontuação F1 para o conjunto de teste: {:.4f}.".format(predict_labels(clf, X_test, y_test))
    

### Implementação: Métricas de Desempenho do Modelo
Com as funções acima, você vai importar os três modelos de aprendizagem supervisionada de sua escolha e executar a função `train_prediction` para cada um deles. Lembre-se de que você vai precisar treinar e usar cada classificador para três diferentes tamanhos de conjuntos de treinamentos: 100, 200 e 300 pontos. Então você deve ter 9 saídas diferentes abaixo – 3 para cada modelo utilizando cada tamanho de conjunto de treinamento. Na célula de código a seguir, você deve implementar o seguinte:
- Importe os três modelos de aprendizagem supervisionada que você escolheu na seção anterior.
- Inicialize os três modelos e armazene eles em `clf_A`, `clf_B` e `clf_C`.
 - Defina um `random_state` para cada modelo, se a opção existir.
 - **Nota:** Utilize as configurações padrão para cada modelo – você vai calibrar um modelo específico em uma seção posterior.
- Crie diferentes tamanhos de conjuntos de treinamento para treinar cada modelo.
 - *Não embaralhe e distribua novamente os dados! Os novos pontos de treinamento devem ser tirados de `X_train` e `y_train`.*
- Treine cada modelo com cada tamanho de conjunto de treinamento e faça estimativas com o conjunto de teste (9 vezes no total).  
**Nota:** Três tabelas são fornecidas depois da célula de código a seguir, nas quais você deve anotar seus resultados.

In [7]:
# TODO: Importe os três modelos de aprendizagem supervisionada do sklearn
from sklearn.ensemble import RandomForestClassifier
#from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

# TODO: Inicialize os três modelos
clf_A = LinearSVC(random_state=42)
clf_B = KNeighborsClassifier()
clf_C = RandomForestClassifier(random_state=42)

for clf in [clf_A, clf_B, clf_C]:
    for n_train in [100, 200, 300]:
        train_predict(clf, 
                      X_train[:n_train], y_train[:n_train], 
                      X_test, y_test)
        print ''



Treinando um LinearSVC com 100 pontos de treinamento. . .
O modelo foi treinado em 0.0112 segundos
As previsões foram feitas em 0.0021 segundos.
Accuracy Score 0.83
Recall Score [ 0.58333333  0.96875   ]
Precision Score 0.805194805195
Pontuação F1 para o conjunto de treino: 0.8794.
As previsões foram feitas em 0.0002 segundos.
Accuracy Score 0.652631578947
Recall Score [ 0.31428571  0.85      ]
Precision Score 0.68
Pontuação F1 para o conjunto de teste: 0.7556.

Treinando um LinearSVC com 200 pontos de treinamento. . .
O modelo foi treinado em 0.0185 segundos
As previsões foram feitas em 0.0003 segundos.
Accuracy Score 0.77
Recall Score [ 0.72580645  0.78985507]
Precision Score 0.865079365079
Pontuação F1 para o conjunto de treino: 0.8258.
As previsões foram feitas em 0.0002 segundos.
Accuracy Score 0.715789473684
Recall Score [ 0.48571429  0.85      ]
Precision Score 0.739130434783
Pontuação F1 para o conjunto de teste: 0.7907.

Treinando um LinearSVC com 300 pontos de treinamento. . 

### Resultados Tabulados
Edite a célula abaixo e veja como a tabela pode ser desenhada em [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). Você deve salvar seus resultados abaixo nas tabelas fornecidas.

** Classificador 2 - LinearSVC**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                | 0.0090               |    0.0010                  | 0.8905                  | 0.7463                     |
| 200                                |       0.0170        |   0.0000                    |  0.8523                  | 0.7737               |
| 300                                |  0.0280            |      0.0000                  |  0.5993                   |       0.6471      |

** Classificador 3 - KNeighborsClassifier**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                | 0.0000               |   0.0020                    |    0.8060                 |  0.7246              |
| 200                                |  0.0010              |   0.0030                    |   0.8800                |  0.7692            |
| 300                                | 0.0010               |  0.0040                     |  0.8809                   | 0.7801             |

** Classificador 3 - RandomForestClassifier**  

| Tamanho do Conjunto de Treinamento | Tempo de Treinamento | Tempo de Estimativa (teste) | Pontuação F1 (treinamento) | Pontuação F1 (teste) |
| :--------------------------------: | :------------------: | :-------------------------: | :------------------------: | :------------------: |
| 100                                |  0.0318              |  0.0053                     |   0.9844                     | 0.7153                     |
| 200                                |       0.0264        |      0.0056              |    0.9964                   | 0.7857                     |
| 300                                |    0.0269           |     0.0054                |   0.9951                   |        0.8092       |

## Escolhendo o Melhor Modelo
Nesta seção final, você irá escolher dos três modelos de aprendizagem supervisionada o *melhor* para utilizar os dados dos estudantes. Você então executará um busca em matriz otimizada para o modelo em todo o conjunto de treinamento (`X_train` e `y_train`) ao calibrar pelo menos um parâmetro, melhorando em comparação a pontuação F<sub>1</sub> do modelo não calibrado. 

### Questão 3 - Escolhendo o Melhor Modelo
*Baseando-se nos experimentos que você executou até agora, explique em um ou dois parágrafos ao conselho de supervisores qual modelo que você escolheu como o melhor. Qual modelo é o mais apropriado baseado nos dados disponíveis, recursos limitados, custo e desempenho?*

**Resposta: **

O modelo que teve o pior desempenho na pontuação F1 dos testes foi LinearSVC. Isso se deve ao fato que provavelmente estes dados não são muito bem divididos linearmente. O tempo de treinamento também aumenta significativamente ao aumentar o conjunto de dados, enquanto que ele é muito rápido em estimar. Apesar do bom desempenho na velocidade de estimar os dados de testes, este não se mostrou um bom modelo, pois a pontuação F1 nos testes foi bem inferior comparando com os outros algoritmos, principalmente no conjunto de 300 pontos.<br>

Levando em consideração o tempo de treinamento, como esperado, o modelo mais rápido é o KNeighborsClassifier, levando 0.0010 segundos no máximo. O tempo de estimativa percebeu-se que aumentou proporcionalmente ao tamanho do conjunto de pontos de testes, caso tivesse muito mais dados, o custo computacional para previsão seria muito alto. A pontuação F1 deste modelo ficou razoavel. <br>

RandomForestClassifier teve o maior tempo de treinamento no conjunto de 100 e 200 pontos, porém em 300 pontos percebe-se que ele tem o desempenho melhor que o LinearSVC. O tempo de testes também foi maior entre os modelos, entretanto a variação de tempo entre o conjunto de dados (100, 200 e 300) não é muito grande. Foi o classificador que teve o **melhor desempenho na pontuação F1**, por isso eu escolho **RandomForestClassifier como melhor modelo**.




### Questão 4 – O Modelo para um Leigo
*Em um ou dois parágrafos, explique para o conselho de supervisores, utilizando termos leigos, como o modelo final escolhido deve trabalhar. Tenha certeza que você esteja descrevendo as melhores qualidades do modelo, por exemplo, como o modelo é treinado e como ele faz uma estimativa. Evite jargões técnicos ou matemáticos, como descrever equações ou discutir a implementação do algoritmo.*

**Resposta: **

As árvores de decisão são fáceis de entender, pois criam regras de decisões.
Exemplo, se (modelo do carro é X) e (cidade de emplacamento y) e (tem z anos de fabricação) e (teve k donos) então (seu preço de venda é 30000). <br>

Tem este nome pois sua estrutura se assemelha a uma árvore, com uma raíz e folhas. <br>

Random Forest (florestas aleatórias) estende as árvores de decisão, incluindo várias árvores, com amostras aleatórias dos dados, combinando de diferentes formas os atributos. <br>
Ou seja, em vez de usar apenas uma árvore, com uma visão, esta abordagem utiliza uma coleção de árvores, cada uma montada utilizando dados aleatórios, com diferentes visões e resultados. <br>

Ao final este modelo combina o resultado de classificação de cada árvore de decisão, garantindo um desempenho melhor do que se fosse utilizar apenas uma árvore.


### Implementação: Calibrando o Modelo
Calibre o modelo escolhido. Utilize busca em matriz (`GridSearchCV`) com, pelo menos, um parâmetro importante calibrado com, pelo menos, 3 valores diferentes. Você vai precisar utilizar todo o conjunto de treinamento para isso. Na célula de código abaixo, você deve implementar o seguinte:
- Importe [`sklearn.grid_search.gridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html) e [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Crie o dicionário de parâmetros que você deseja calibrar para o modelo escolhido.
 - Examplo: `parameters = {'parameter' : [list of values]}`.
- Inicialize o classificador que você escolheu e armazene-o em `clf`.
- Crie a função de pontuação F<sub>1</sub> utilizando `make_scorer` e armazene-o em `f1_scorer`.
 - Estabeleça o parâmetro `pos_label` para o valor correto!
- Execute uma busca em matriz no classificador `clf` utilizando o `f1_scorer` como método de pontuação e armazene-o em `grid_obj`.
- Treine o objeto de busca em matriz com os dados de treinamento (`X_train`, `y_train`) e armazene-o em `grid_obj`.

In [8]:
# TODO: Importe 'GridSearchCV' e 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


# TODO: Crie a lista de parâmetros que você gostaria de calibrar
parameters =  {'n_estimators':[10,500,600,700,800,900,1000,1500],
               'max_features':[15,30,40,n_features]}

# TODO: Inicialize o classificador
clf = RandomForestClassifier(random_state=42)



# TODO: Faça uma função de pontuação f1 utilizando 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label='yes')

# TODO: Execute uma busca em matriz no classificador utilizando o f1_scorer como método de pontuação
grid_obj = GridSearchCV(estimator=clf, param_grid=parameters, scoring=f1_scorer, verbose=1)

# TODO: Ajuste o objeto de busca em matriz para o treinamento de dados e encontre os parâmetros ótimos
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

print clf

# Reporte a pontuação final F1 para treinamento e teste depois de calibrar os parâmetrosprint "Tuned model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de treinamento.".format(predict_labels(clf, X_train, y_train))
print "O modelo calibrado tem F1 de {:.4f} no conjunto de teste.".format(predict_labels(clf, X_test, y_test))

importances = clf.feature_importances_
std = np.std([tree.feature_importances_ for tree in clf.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

# Print the feature ranking
print("Ranking de Atributos:")

for f in range(X_train.shape[1]):
    print("{}. atributo {}-{} ({})".format(f + 1, indices[f], X_all.columns[indices[f]], importances[indices[f]]))



Fitting 3 folds for each of 32 candidates, totalling 96 fits


[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:  4.5min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=30, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=900, n_jobs=1, oob_score=False, random_state=42,
            verbose=0, warm_start=False)
As previsões foram feitas em 0.5329 segundos.
Accuracy Score 1.0
Recall Score [ 1.  1.]
Precision Score 1.0
O modelo calibrado tem F1 de 1.0000 no conjunto de treinamento.
As previsões foram feitas em 0.4363 segundos.
Accuracy Score 0.726315789474
Recall Score [ 0.34285714  0.95      ]
Precision Score 0.7125
O modelo calibrado tem F1 de 0.8143 no conjunto de teste.
Ranking de Atributos:
1. atributo 47-absences (0.117940879872)
2. atributo 32-failures (0.0962996021213)
3. atributo 43-goout (0.0588489366795)
4. atributo 4-age (0.0521122155093)
5. atributo 42-freetime (0.0391359207015)
6. atributo 46-health (0.0383188

### Questão 5 - Pontuação F<sub>1</sub> Final
*Qual é a pontuação F<sub>1</sub> do modelo final para treinamento e teste? Como ele se compara ao modelo que não foi calibrado?*

**Resposta: **

A pontuação F1 para o modelo calibrado ficou em 0.8143, já o modelo não calibrado em 300 pontos ficou com pontuação F1 0.8092. Foi possível perceber uma melhora no modelo calibrado em 0.63%.


> **Nota**: Uma vez que você completou todas as implementações de código e respondeu todas as questões acima com êxito, você pode finalizar seu trabalho exportando o iPython Nothebook como um document HTML. Você pode fazer isso utilizando o menu acima e navegando para  
**File -> Download as -> HTML (.html)**. Inclua a documentação final junto com o notebook para o envio do seu projeto.